In [1]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt

######################################################################################################################
import sys
import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode

plt.style.use('bmh')
%matplotlib inline

try:
    from IPython.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

class KnnDtw(object):
    """K-nearest neighbor classifier using dynamic time warping
    as the distance measure between pairs of time series arrays
    
    Arguments
    ---------
    n_neighbors : int, optional (default = 5)
        Number of neighbors to use by default for KNN
        
    max_warping_window : int, optional (default = infinity)
        Maximum warping window allowed by the DTW dynamic
        programming function
            
    subsample_step : int, optional (default = 1)
        Step size for the timeseries array. By setting subsample_step = 2,
        the timeseries length will be reduced by 50% because every second
        item is skipped. Implemented by x[:, ::subsample_step]
    """
    
    def __init__(self, n_neighbors=5, max_warping_window=10000, subsample_step=1):
        self.n_neighbors = n_neighbors
        self.max_warping_window = max_warping_window
        self.subsample_step = subsample_step
    
    def fit(self, x, l):
        """Fit the model using x as training data and l as class labels
        
        Arguments
        ---------
        x : array of shape [n_samples, n_timepoints]
            Training data set for input into KNN classifer
            
        l : array of shape [n_samples]
            Training labels for input into KNN classifier
        """
        
        self.x = x
        self.l = l
        
    def _dtw_distance(self, ts_a, ts_b, d = lambda x,y: abs(x-y)):
        """Returns the DTW similarity distance between two 2-D
        timeseries numpy arrays.

        Arguments
        ---------
        ts_a, ts_b : array of shape [n_samples, n_timepoints]
            Two arrays containing n_samples of timeseries data
            whose DTW distance between each sample of A and B
            will be compared
        
        d : DistanceMetric object (default = abs(x-y))
            the distance measure used for A_i - B_j in the
            DTW dynamic programming function
        
        Returns
        -------
        DTW distance between A and B
        """

        # Create cost matrix via broadcasting with large int
        ts_a, ts_b = np.array(ts_a), np.array(ts_b)
        M, N = len(ts_a), len(ts_b)
        cost = sys.maxsize * np.ones((M, N))

        # Initialize the first row and column
        cost[0, 0] = d(ts_a[0], ts_b[0])
        for i in range(1, M):
            cost[i, 0] = cost[i-1, 0] + d(ts_a[i], ts_b[0])

        for j in range(1, N):
            cost[0, j] = cost[0, j-1] + d(ts_a[0], ts_b[j])

        # Populate rest of cost matrix within window
        for i in range(1, M):
            for j in range(max(1, i - self.max_warping_window),
                            min(N, i + self.max_warping_window)):
                choices = cost[i - 1, j - 1], cost[i, j-1], cost[i-1, j]
                cost[i, j] = min(choices) + d(ts_a[i], ts_b[j])

        # Return DTW distance given window 
        return cost[-1, -1]
    
    def _dist_matrix(self, x, y):
        """Computes the M x N distance matrix between the training
        dataset and testing dataset (y) using the DTW distance measure
        
        Arguments
        ---------
        x : array of shape [n_samples, n_timepoints]
        
        y : array of shape [n_samples, n_timepoints]
        
        Returns
        -------
        Distance matrix between each item of x and y with
            shape [training_n_samples, testing_n_samples]
        """
        
        # Compute the distance matrix        
        dm_count = 0
        
        # Compute condensed distance matrix (upper triangle) of pairwise dtw distances
        # when x and y are the same array
        if(np.array_equal(x, y)):
            x_s = shape(x)
            dm = np.zeros((x_s[0] * (x_s[0] - 1)) // 2, dtype=np.double)
            
            #p = ProgressBar(shape(dm)[0])
            
            for i in range(0, x_s[0] - 1):
                for j in range(i + 1, x_s[0]):
                    dm[dm_count] = self._dtw_distance(x[i, ::self.subsample_step],
                                                      y[j, ::self.subsample_step])
                    
                    dm_count += 1
                    #p.animate(dm_count)
            
            # Convert to squareform
            dm = squareform(dm)
            return dm
        
        # Compute full distance matrix of dtw distnces between x and y
        else:
            x_s = np.shape(x)
            y_s = np.shape(y)
            dm = np.zeros((x_s[0], y_s[0])) 
            dm_size = x_s[0]*y_s[0]
            
            #p = ProgressBar(dm_size)
        
            for i in range(0, x_s[0]):
                for j in range(0, y_s[0]):
                    dm[i, j] = self._dtw_distance(x[i, ::self.subsample_step],
                                                  y[j, ::self.subsample_step])
                    # Update progress bar
                    dm_count += 1
                    #p.animate(dm_count)
            return dm
        
    def predict(self, x):
        """Predict the class labels or probability estimates for 
        the provided data

        Arguments
        ---------
          x : array of shape [n_samples, n_timepoints]
              Array containing the testing data set to be classified
          
        Returns
        -------
          2 arrays representing:
              (1) the predicted class labels 
              (2) the knn label count probability
        """
        
        dm = self._dist_matrix(x, self.x)

        # Identify the k nearest neighbors
        knn_idx = dm.argsort()[:, :self.n_neighbors]

        # Identify k nearest labels
        knn_labels = self.l[knn_idx]
        
        # Model Label
        mode_data = mode(knn_labels, axis=1)
        mode_label = mode_data[0]
        mode_proba = mode_data[1]/self.n_neighbors

        return mode_label.ravel(), mode_proba.ravel()

class ProgressBar:
    """This progress bar was taken from PYMC
    """
    def __init__(self, iterations):
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 40
        self.__update_amount(0)
        if have_ipython:
            self.animate = self.animate_ipython
        else:
            self.animate = self.animate_noipython

    def animate_ipython(self, iter):
        sys.stdout.write('\r%s'%self)
        sys.stdout.flush()
        self.update_iteration(iter + 1)

    def update_iteration(self, elapsed_iter):
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])

    def __str__(self):
        return str(self.prog_bar)

######################################################################################################################


import tensorflow as tf 
import keras 
import keras.backend as K

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from collections import Counter

from keras import regularizers
from keras.models import Sequential, Model, load_model, model_from_json 
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Reshape, Concatenate,  Dropout 
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU
from scipy.signal import resample


################################################################################
def get_ds_infos():
    """
    Read the file includes data subject information.
    
    Data Columns:
    0: code [1-24]
    1: weight [kg]
    2: height [cm]
    3: age [years]
    4: gender [0:Female, 1:Male]
    
    Returns:
        A pandas DataFrame that contains inforamtion about data subjects' attributes 
    """ 

    dss = pd.read_csv("data_subjects_info.csv")
    print("[INFO] -- Data subjects' information is imported.")
    
    return dss

def set_data_types(data_types=["userAcceleration"]):
    """
    Select the sensors and the mode to shape the final dataset.
    
    Args:
        data_types: A list of sensor data type from this list: [attitude, gravity, rotationRate, userAcceleration] 

    Returns:
        It returns a list of columns to use for creating time-series from files.
    """
    dt_list = []
    for t in data_types:
        if t != "attitude":
            dt_list.append([t+".x",t+".y",t+".z"])
        else:
            dt_list.append([t+".roll", t+".pitch", t+".yaw"])

    return dt_list


def creat_time_series(dt_list, act_labels, trial_codes, mode="mag", labeled=True, combine_grav_acc=False):
    """
    Args:
        dt_list: A list of columns that shows the type of data we want.
        act_labels: list of activites
        trial_codes: list of trials
        mode: It can be "raw" which means you want raw data
        for every dimention of each data type,
        [attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)].
        or it can be "mag" which means you only want the magnitude for each data type: (x^2+y^2+z^2)^(1/2)
        labeled: True, if we want a labeld dataset. False, if we only want sensor values.
        combine_grav_acc: True, means adding each axis of gravity to  corresponding axis of userAcceleration.
    Returns: 
        It returns a time-series of sensor data.
    
    """
    num_data_cols = len(dt_list) if mode == "mag" else len(dt_list*3)

    if labeled:
        dataset = np.zeros((0,num_data_cols+7)) # "7" --> [act, code, weight, height, age, gender, trial] 
    else:
        dataset = np.zeros((0,num_data_cols))
        
    ds_list = get_ds_infos()
    
    print("[INFO] -- Creating Time-Series")
    for sub_id in ds_list["code"]:
        for act_id, act in enumerate(act_labels):
            for trial in trial_codes[act_id]:
                fname = 'A_DeviceMotion_data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                raw_data = pd.read_csv(fname)
                raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
                vals = np.zeros((len(raw_data), num_data_cols))
                
                if combine_grav_acc:
                    raw_data["userAcceleration.x"] = raw_data["userAcceleration.x"].add(raw_data["gravity.x"])
                    raw_data["userAcceleration.y"] = raw_data["userAcceleration.y"].add(raw_data["gravity.y"])
                    raw_data["userAcceleration.z"] = raw_data["userAcceleration.z"].add(raw_data["gravity.z"])
                
                for x_id, axes in enumerate(dt_list):
                    if mode == "mag":
                        vals[:,x_id] = (raw_data[axes]**2).sum(axis=1)**0.5        
                    else:
                        vals[:,x_id*3:(x_id+1)*3] = raw_data[axes].values
                    vals = vals[:,:num_data_cols]
                if labeled:
                    lbls = np.array([[act_id,
                            sub_id-1,
                            ds_list["weight"][sub_id-1],
                            ds_list["height"][sub_id-1],
                            ds_list["age"][sub_id-1],
                            ds_list["gender"][sub_id-1],
                            trial          
                           ]]*len(raw_data))
                    vals = np.concatenate((vals, lbls), axis=1)
                dataset = np.append(dataset,vals, axis=0)
    cols = []
    for axes in dt_list:
        if mode == "raw":
            cols += axes
        else:
            cols += [str(axes[0][:-2])]
            
    if labeled:
        cols += ["act", "id", "weight", "height", "age", "gender", "trial"]
    
    dataset = pd.DataFrame(data=dataset, columns=cols)
    return dataset
#________________________________
#________________________________

def ts_to_secs(dataset, w, s, standardize = False, **options):
    
    data = dataset[dataset.columns[:-7]].values    
    act_labels = dataset["act"].values
    id_labels = dataset["id"].values
    trial_labels = dataset["trial"].values

    mean = 0
    std = 1
    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            std = options.get("std")
            print("[INFO] -- Test Data has been standardized")
        else:
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("[INFO] -- Training Data has been standardized: the mean is = "+str(mean)+" ; and the std is = "+str(std))            

        data -= mean
        data /= std
    else:
        print("[INFO] -- Without Standardization.....")

    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T

    m = data.shape[0]   # Data Dimension 
    ttp = data.shape[1] # Total Time Points
    number_of_secs = int(round(((ttp - w)/s)))

    ##  Create a 3D matrix for Storing Sections  
    secs_data = np.zeros((number_of_secs , m , w ))
    act_secs_labels = np.zeros(number_of_secs)
    id_secs_labels = np.zeros(number_of_secs)

    k=0
    for i in range(0 , ttp-w, s):
        j = i // s
        if j >= number_of_secs:
            break
        if id_labels[i] != id_labels[i+w-1]: 
            continue
        if act_labels[i] != act_labels[i+w-1]: 
            continue
        if trial_labels[i] != trial_labels[i+w-1]:
            continue
            
        secs_data[k] = data[:, i:i+w]
        act_secs_labels[k] = act_labels[i].astype(int)
        id_secs_labels[k] = id_labels[i].astype(int)
        k = k+1
        
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    id_secs_labels = id_secs_labels[0:k]
    return secs_data, act_secs_labels, id_secs_labels, mean, std
##________________________________________________________________


ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]
TRIAL_CODES = {
    ACT_LABELS[0]:[1,2,11],
    ACT_LABELS[1]:[3,4,12],
    ACT_LABELS[2]:[7,8,15],
    ACT_LABELS[3]:[9,16],
    ACT_LABELS[4]:[6,14],
    ACT_LABELS[5]:[5,13],
}

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["rotationRate","userAcceleration"]
mode = "mag"
cga = True # Add gravity to acceleration or not
print("[INFO] -- Selected sensor data types: "+str(sdt)+" -- Mode: "+str(mode)+" -- Grav+Acc: "+str(cga))    

act_labels = ACT_LABELS [0:4]
print("[INFO] -- Selected activites: "+str(act_labels))    
trial_codes = [TRIAL_CODES[act] for act in act_labels]
dt_list = set_data_types(sdt)
dataset = creat_time_series(dt_list, act_labels, trial_codes, mode=mode, labeled=True, combine_grav_acc = cga)
print("[INFO] -- Shape of time-Series dataset:"+str(dataset.shape))    

#*****************
TRAIN_TEST_TYPE = "subject" # "subject" or "trial"
#*****************

if TRAIN_TEST_TYPE == "subject":
    test_ids = [4,9,11,21]
    print("[INFO] -- Test IDs: "+str(test_ids))
    test_ts = dataset.loc[(dataset['id'].isin(test_ids))]
    train_ts = dataset.loc[~(dataset['id'].isin(test_ids))]
else:
    test_trail = [11,12,13,14,15,16]  
    print("[INFO] -- Test Trials: "+str(test_trail))
    test_ts = dataset.loc[(dataset['trial'].isin(test_trail))]
    train_ts = dataset.loc[~(dataset['trial'].isin(test_trail))]

    
print("[INFO] -- Shape of Train Time-Series :"+str(train_ts.shape))
print("[INFO] -- Shape of Test Time-Series :"+str(test_ts.shape))


[INFO] -- Selected sensor data types: ['rotationRate', 'userAcceleration'] -- Mode: mag -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 9)
[INFO] -- Test IDs: [4, 9, 11, 21]
[INFO] -- Shape of Train Time-Series :(646207, 9)
[INFO] -- Shape of Test Time-Series :(121453, 9)


In [3]:
val_trail = [11,12,13,14,15,16]
val_ts = train_ts.loc[(train_ts['trial'].isin(val_trail))]
train_ts = train_ts.loc[~(train_ts['trial'].isin(val_trail))]
print("[INFO] -- Shape of Train Time-Series :"+str(train_ts.shape))
print("[INFO] -- Shape of Test Time-Series :"+str(val_ts.shape))

[INFO] -- Shape of Train Time-Series :(523129, 9)
[INFO] -- Shape of Test Time-Series :(123078, 9)


In [4]:
#************
## HERE ##

## This Variable Defines the Size of Sliding Window
## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
w = 128 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
s = 10
train_data, act_train, id_train, train_mean, train_std = ts_to_secs(train_ts.copy(),
                                                                   w,
                                                                   s,
                                                                   standardize = True)

s = 10
val_data, act_val, id_val, val_mean, val_std = ts_to_secs(val_ts.copy(),
                                                              w,
                                                              s,
                                                              standardize = True,
                                                              mean = train_mean, 
                                                              std = train_std)


s = 10
test_data, act_test, id_test, test_mean, test_std = ts_to_secs(test_ts.copy(),
                                                              w,
                                                              s,
                                                              standardize = True,
                                                              mean = train_mean, 
                                                              std = train_std)

print("[INFO] -- Shape of Training Sections: "+str(train_data.shape))
print("[INFO] -- Shape of Training Sections: "+str(val_data.shape))
print("[INFO] -- Shape of Test Sections:  "+str(test_data.shape))

[INFO] -- Training Data has been standardized: the mean is = [2.17728825 1.19431016] ; and the std is = [1.43229632 0.70168121]
[INFO] -- Test Data has been standardized
[INFO] -- Test Data has been standardized
[INFO] -- Shape of Training Sections: (50532, 2, 128)
[INFO] -- Shape of Training Sections: (11288, 2, 128)
[INFO] -- Shape of Test Sections:  (11589, 2, 128)


In [5]:
id_train_labels = to_categorical(id_train)
id_val_labels = to_categorical(id_val)
id_test_labels = to_categorical(id_test)
id_test_labels = np.append(id_test_labels, np.zeros((len(id_test_labels),2)), axis =1)

act_train_labels = to_categorical(act_train)
act_val_labels = to_categorical(act_val)
act_test_labels = to_categorical(act_test)
    
## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
train_data = np.expand_dims(train_data,axis=3)
print("[INFO] -- Shape of Training Sections:", train_data.shape)
val_data = np.expand_dims(val_data,axis=3)
print("[INFO] -- Validation Sections:"+str(val_data.shape))
test_data = np.expand_dims(test_data,axis=3)
print("[INFO] -- Shape of Training Sections:", test_data.shape)

[INFO] -- Shape of Training Sections: (50532, 2, 128, 1)
[INFO] -- Validation Sections:(11288, 2, 128, 1)
[INFO] -- Shape of Training Sections: (11589, 2, 128, 1)


In [6]:
Xraw = train_data.copy()
tXraw = test_data.copy()
test_ids = [4,9,11,21]
#*******************
sample_rate = 10 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("Number of Sampels = "+str(num_sampels))
tX10 = tXraw.copy()

from scipy.signal import resample
ds_test_data = tX10.copy()

for sens in range(2):
    tmp = np.array([resample(x,num_sampels) for x in ds_test_data[:,sens,:,0]])
    ds_test_data[:,sens,:num_sampels,0] = tmp 

ds_test_data = ds_test_data[:,:,:num_sampels,:]
print("[INFO] -- Downsampled Test Sections:", ds_test_data.shape)

for sens in range(2):
    tmp = np.array([resample(x,128) for x in ds_test_data[:,sens,:,0]])
    tX10[:,sens,:,0] = tmp 
    
print("[INFO] -- Test Sections:", tX10.shape)
print(tX10.shape)

Number of Sampels = 25
[INFO] -- Downsampled Test Sections: (11589, 2, 25, 1)
[INFO] -- Test Sections: (11589, 2, 128, 1)
(11589, 2, 128, 1)


In [7]:
dist = np.zeros((4,24))
ranks = np.zeros(4)
kwd = KnnDtw(n_neighbors=10, subsample_step=2)
nn = {} 
mean = 0
std = 0
dd = (w//s)*4

## Labels
rdu_act_X = act_train_labels[::dd]
rdu_act_tX = act_test_labels[::dd]
rdu_id_X = id_train_labels[::dd]
rdu_id_tX = id_test_labels[::dd]

# Test Data: after Transformation
rdu_tX10 = tX10[::dd]
print(rdu_tX10.shape)


# Training and Test Data: Raw
rdu_Xraw = Xraw[::dd]
rdu_tXraw = tXraw[::dd]
rdu_all_Xraw = np.append(rdu_Xraw, rdu_tXraw, axis=0)
rdu_all_act = np.append(rdu_act_X,rdu_act_tX,axis=0)
rdu_all_id = np.append(rdu_id_X,rdu_id_tX,axis=0)


import sys
# just considering one activity, like walking. We want to compare data acrros users not activites. 
for act in range(4):
    dic = {}
    for i in range(len(test_ids)):
        print("Test User: "+str(test_ids[i]))
        # Transformed Data
        Xhi = rdu_tX10[np.logical_and(rdu_id_tX[:,test_ids[i]]==1., rdu_act_tX[:,act]==1.)]
        for j in range(24):
            # Raw Data
            Xj = rdu_all_Xraw[np.logical_and(rdu_all_id[:,j]==1., rdu_all_act[:,act]==1.)]
            mat = kwd._dist_matrix(Xhi[:,0,:,0], Xj[:,0,:,0])
            s0 = (mat.sum())/(len(Xhi)*len(Xj))
            
            mat = kwd._dist_matrix(Xhi[:,1,:,0], Xj[:,1,:,0])
            s1 = (mat.sum())/(len(Xhi)*len(Xj))
            dist[i,j] = (s0+s1)/2.
            
            sys.stdout.write("\rj: "+str(j))
            sys.stdout.flush()

        print("")    
        tmp = np.argsort(dist[i])
        tmp = np.where(tmp == test_ids[i])
        ranks[i] = tmp[0][0]
        print("Rank: "+str(ranks[i]))
        if np.argsort(dist[i])[0] in nn:
            nn[np.argsort(dist[i])[0]] += 1
        else:
            nn[np.argsort(dist[i])[0]] = 1 
    print(nn)
    print(ranks)  
    mean += ranks.mean()
    std +=ranks.std()
    print("Mean: "+str(mean/(act+1))+" -- Std = "+str(std/(act+1)))
mean = mean/4.
std = std/4.
print("Final Mean: "+str(mean)+" -- Final Std = "+str(std))

(242, 2, 128, 1)
Test User: 4
j: 23
Rank: 2.0
Test User: 9
j: 23
Rank: 5.0
Test User: 11
j: 23
Rank: 15.0
Test User: 21
j: 23
Rank: 11.0
{2: 1, 19: 3}
[ 2.  5. 15. 11.]
Mean: 8.25 -- Std = 5.0682837331783235
Test User: 4
j: 23
Rank: 0.0
Test User: 9
j: 23
Rank: 12.0
Test User: 11
j: 23
Rank: 2.0
Test User: 21
j: 23
Rank: 13.0
{2: 1, 19: 3, 4: 1, 10: 1, 22: 2}
[ 0. 12.  2. 13.]
Mean: 7.5 -- Std = 5.436188558150137
Test User: 4
j: 23
Rank: 0.0
Test User: 9
j: 23
Rank: 14.0
Test User: 11
j: 23
Rank: 15.0
Test User: 21
j: 23
Rank: 5.0
{2: 1, 19: 3, 4: 2, 22: 4, 20: 1, 10: 1}
[ 0. 14. 15.  5.]
Mean: 7.833333333333333 -- Std = 5.712453053123703
Test User: 4
j: 23
Rank: 0.0
Test User: 9
j: 23
Rank: 14.0
Test User: 11
j: 23
Rank: 7.0
Test User: 21
j: 23
Rank: 0.0
{2: 1, 19: 3, 4: 5, 21: 1, 22: 4, 20: 1, 10: 1}
[ 0. 14.  7.  0.]
Mean: 7.1875 -- Std = 5.735363135623265
Final Mean: 7.1875 -- Final Std = 5.735363135623265


In [8]:
Xraw = train_data.copy()
tXraw = test_data.copy()
test_ids = [4,9,11,21]
#*******************
sample_rate = 5 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("Number of Sampels = "+str(num_sampels))
tX10 = tXraw.copy()

from scipy.signal import resample
ds_test_data = tX10.copy()

for sens in range(2):
    tmp = np.array([resample(x,num_sampels) for x in ds_test_data[:,sens,:,0]])
    ds_test_data[:,sens,:num_sampels,0] = tmp 

ds_test_data = ds_test_data[:,:,:num_sampels,:]
print("[INFO] -- Downsampled Test Sections:", ds_test_data.shape)

for sens in range(2):
    tmp = np.array([resample(x,128) for x in ds_test_data[:,sens,:,0]])
    tX10[:,sens,:,0] = tmp 
    
print("[INFO] -- Test Sections:", tX10.shape)
print(tX10.shape)

Number of Sampels = 12
[INFO] -- Downsampled Test Sections: (11589, 2, 12, 1)
[INFO] -- Test Sections: (11589, 2, 128, 1)
(11589, 2, 128, 1)


In [9]:
dist = np.zeros((4,24))
ranks = np.zeros(4)
kwd = KnnDtw(n_neighbors=10, subsample_step=2)
nn = {} 
mean = 0
std = 0
dd = (w//s)*4

## Labels
rdu_act_X = act_train_labels[::dd]
rdu_act_tX = act_test_labels[::dd]
rdu_id_X = id_train_labels[::dd]
rdu_id_tX = id_test_labels[::dd]

# Test Data: after Transformation
rdu_tX10 = tX10[::dd]
print(rdu_tX10.shape)


# Training and Test Data: Raw
rdu_Xraw = Xraw[::dd]
rdu_tXraw = tXraw[::dd]
rdu_all_Xraw = np.append(rdu_Xraw, rdu_tXraw, axis=0)
rdu_all_act = np.append(rdu_act_X,rdu_act_tX,axis=0)
rdu_all_id = np.append(rdu_id_X,rdu_id_tX,axis=0)


import sys
# just considering one activity, like walking. We want to compare data acrros users not activites. 
for act in range(4):
    dic = {}
    for i in range(len(test_ids)):
        print("Test User: "+str(test_ids[i]))
        # Transformed Data
        Xhi = rdu_tX10[np.logical_and(rdu_id_tX[:,test_ids[i]]==1., rdu_act_tX[:,act]==1.)]
        for j in range(24):
            # Raw Data
            Xj = rdu_all_Xraw[np.logical_and(rdu_all_id[:,j]==1., rdu_all_act[:,act]==1.)]
            mat = kwd._dist_matrix(Xhi[:,0,:,0], Xj[:,0,:,0])
            s0 = (mat.sum())/(len(Xhi)*len(Xj))
            
            mat = kwd._dist_matrix(Xhi[:,1,:,0], Xj[:,1,:,0])
            s1 = (mat.sum())/(len(Xhi)*len(Xj))
            dist[i,j] = (s0+s1)/2.
            
            sys.stdout.write("\rj: "+str(j))
            sys.stdout.flush()

        print("")    
        tmp = np.argsort(dist[i])
        tmp = np.where(tmp == test_ids[i])
        ranks[i] = tmp[0][0]
        print("Rank: "+str(ranks[i]))
        if np.argsort(dist[i])[0] in nn:
            nn[np.argsort(dist[i])[0]] += 1
        else:
            nn[np.argsort(dist[i])[0]] = 1 
    print(nn)
    print(ranks)  
    mean += ranks.mean()
    std +=ranks.std()
    print("Mean: "+str(mean/(act+1))+" -- Std = "+str(std/(act+1)))
mean = mean/4.
std = std/4.
print("Final Mean: "+str(mean)+" -- Final Std = "+str(std))

(242, 2, 128, 1)
Test User: 4
j: 23
Rank: 2.0
Test User: 9
j: 23
Rank: 6.0
Test User: 11
j: 23
Rank: 16.0
Test User: 21
j: 23
Rank: 12.0
{2: 2, 19: 2}
[ 2.  6. 16. 12.]
Mean: 9.0 -- Std = 5.385164807134504
Test User: 4
j: 23
Rank: 1.0
Test User: 9
j: 23
Rank: 15.0
Test User: 11
j: 23
Rank: 6.0
Test User: 21
j: 23
Rank: 18.0
{22: 1, 2: 2, 19: 2, 10: 1, 6: 2}
[ 1. 15.  6. 18.]
Mean: 9.5 -- Std = 6.102127827813716
Test User: 4
j: 23
Rank: 7.0
Test User: 9
j: 23
Rank: 14.0
Test User: 11
j: 23
Rank: 14.0
Test User: 21
j: 23
Rank: 6.0
{2: 2, 19: 2, 20: 1, 6: 3, 22: 3, 10: 1}
[ 7. 14. 14.  6.]
Mean: 9.75 -- Std = 5.323628482985757
Test User: 4
j: 23
Rank: 5.0
Test User: 9
j: 23
Rank: 20.0
Test User: 11
j: 23
Rank: 7.0
Test User: 21
j: 23
Rank: 0.0
{2: 2, 19: 2, 20: 1, 6: 3, 22: 3, 10: 1, 18: 1, 21: 3}
[ 5. 20.  7.  0.]
Mean: 9.3125 -- Std = 5.8383242447684935
Final Mean: 9.3125 -- Final Std = 5.8383242447684935


In [10]:
class SSA(object):
    
    __supported_types = (pd.Series, np.ndarray, list)
    
    def __init__(self, tseries, L, save_mem=True):
        """
        Decomposes the given time series with a singular-spectrum analysis. Assumes the values of the time series are
        recorded at equal intervals.
        
        Parameters
        ----------
        tseries : The original time series, in the form of a Pandas Series, NumPy array or list. 
        L : The window length. Must be an integer 2 <= L <= N/2, where N is the length of the time series.
        save_mem : Conserve memory by not retaining the elementary matrices. Recommended for long time series with
            thousands of values. Defaults to True.
        
        Note: Even if an NumPy array or list is used for the initial time series, all time series returned will be
        in the form of a Pandas Series or DataFrame object.
        """
        
        # Tedious type-checking for the initial time series
        if not isinstance(tseries, self.__supported_types):
            raise TypeError("Unsupported time series object. Try Pandas Series, NumPy array or list.")
        
        # Checks to save us from ourselves
        self.N = len(tseries)
        if not 2 <= L <= self.N/2:
            raise ValueError("The window length must be in the interval [2, N/2].")
        
        self.L = L
        self.orig_TS = pd.Series(tseries)
        self.K = self.N - self.L + 1
        
        # Embed the time series in a trajectory matrix
        self.X = np.array([self.orig_TS.values[i:L+i] for i in range(0, self.K)]).T
        
        # Decompose the trajectory matrix
        self.U, self.Sigma, VT = np.linalg.svd(self.X)
        self.d = np.linalg.matrix_rank(self.X)
        
        self.TS_comps = np.zeros((self.N, self.d))
        
        if not save_mem:
            # Construct and save all the elementary matrices
            self.X_elem = np.array([ self.Sigma[i]*np.outer(self.U[:,i], VT[i,:]) for i in range(self.d) ])

            # Diagonally average the elementary matrices, store them as columns in array.           
            for i in range(self.d):
                X_rev = self.X_elem[i, ::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.V = VT.T
        else:
            # Reconstruct the elementary matrices without storing them
            for i in range(self.d):
                X_elem = self.Sigma[i]*np.outer(self.U[:,i], VT[i,:])
                X_rev = X_elem[::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.X_elem = "Re-run with save_mem=False to retain the elementary matrices."
            
            # The V array may also be very large under these circumstances, so we won't keep it.
            self.V = "Re-run with save_mem=False to retain the V matrix."
        
        # Calculate the w-correlation matrix.
        self.calc_wcorr()
            
    def components_to_df(self, n=0):
        """
        Returns all the time series components in a single Pandas DataFrame object.
        """
        if n > 0:
            n = min(n, self.d)
        else:
            n = self.d
        
        # Create list of columns - call them F0, F1, F2, ...
        cols = ["F{}".format(i) for i in range(n)]
        return pd.DataFrame(self.TS_comps[:, :n], columns=cols, index=self.orig_TS.index)
            
    
    def reconstruct(self, indices):
        """
        Reconstructs the time series from its elementary components, using the given indices. Returns a Pandas Series
        object with the reconstructed time series.
        
        Parameters
        ----------
        indices: An integer, list of integers or slice(n,m) object, representing the elementary components to sum.
        """
        if isinstance(indices, int): indices = [indices]
        
        ts_vals = self.TS_comps[:,indices].sum(axis=1)
        return pd.Series(ts_vals, index=self.orig_TS.index)
    
    def calc_wcorr(self):
        """
        Calculates the w-correlation matrix for the time series.
        """
             
        # Calculate the weights
        w = np.array(list(np.arange(self.L)+1) + [self.L]*(self.K-self.L-1) + list(np.arange(self.L)+1)[::-1])
        
        def w_inner(F_i, F_j):
            return w.dot(F_i*F_j)
        
        # Calculated weighted norms, ||F_i||_w, then invert.
        F_wnorms = np.array([w_inner(self.TS_comps[:,i], self.TS_comps[:,i]) for i in range(self.d)])
        F_wnorms = F_wnorms**-0.5
        
        # Calculate Wcorr.
        self.Wcorr = np.identity(self.d)
        for i in range(self.d):
            for j in range(i+1,self.d):
                self.Wcorr[i,j] = abs(w_inner(self.TS_comps[:,i], self.TS_comps[:,j]) * F_wnorms[i] * F_wnorms[j])
                self.Wcorr[j,i] = self.Wcorr[i,j]
    
    def plot_wcorr(self, min=None, max=None):
        """
        Plots the w-correlation matrix for the decomposed time series.
        """
        if min is None:
            min = 0
        if max is None:
            max = self.d
        
        if self.Wcorr is None:
            self.calc_wcorr()
        
        ax = plt.imshow(self.Wcorr,interpolation = 'none')
        plt.xlabel(r"$\tilde{F}_i$")
        plt.ylabel(r"$\tilde{F}_j$")
        plt.colorbar(ax.colorbar, fraction=0.045)
        ax.colorbar.set_label("$W_{i,j}$")
        plt.clim(0,1)
        
        # For plotting purposes:
        if max == self.d:
            max_rnge = self.d-1
        else:
            max_rnge = max
        
        plt.xlim(min-0.5, max_rnge+0.5)
        plt.ylim(max_rnge+0.5, min-0.5)
           

In [11]:
import sys
window = 10 # SSA window == number of components
ssa_test_data = test_data.copy()
ssa_test_0 = []
ssa_test_1 = []

print("\nTest \n")
for i in range(len(ssa_test_data)):
    ssa_test_0.append(SSA(ssa_test_data[i,0,:,0], window))
    ssa_test_1.append(SSA(ssa_test_data[i,1,:,0], window))
    if(i%100==1):
        sys.stdout.write("\rNow: "+str(np.round(i*100/len(ssa_test_data), 2))+"%")
        sys.stdout.flush()


Test 

Now: 99.24%

In [12]:
tX = test_data.copy()

num_comps = 1
print("With "+str(num_comps)+" components:")
 
for i in range(len(tX)):
    tX[i,0,:,0] = ssa_test_0[i].reconstruct(list(range(0,num_comps)))
    tX[i,1,:,0] = ssa_test_1[i].reconstruct(list(range(0,num_comps)))


With 1 components:


In [13]:
dist = np.zeros((4,24))
ranks = np.zeros(4)
kwd = KnnDtw(n_neighbors=10, subsample_step=2)
nn = {} 
mean = 0
std = 0
dd = (w//s)*4

## Labels
rdu_act_X = act_train_labels[::dd]
rdu_act_tX = act_test_labels[::dd]
rdu_id_X = id_train_labels[::dd]
rdu_id_tX = id_test_labels[::dd]

# Test Data: after Transformation
rdu_tX10 = tX[::dd]
print(rdu_tX10.shape)


# Training and Test Data: Raw
rdu_Xraw = Xraw[::dd]
rdu_tXraw = tXraw[::dd]
rdu_all_Xraw = np.append(rdu_Xraw, rdu_tXraw, axis=0)
rdu_all_act = np.append(rdu_act_X,rdu_act_tX,axis=0)
rdu_all_id = np.append(rdu_id_X,rdu_id_tX,axis=0)


import sys
# just considering one activity, like walking. We want to compare data acrros users not activites. 
for act in range(4):
    dic = {}
    for i in range(len(test_ids)):
        print("Test User: "+str(test_ids[i]))
        # Transformed Data
        Xhi = rdu_tX10[np.logical_and(rdu_id_tX[:,test_ids[i]]==1., rdu_act_tX[:,act]==1.)]
        for j in range(24):
            # Raw Data
            Xj = rdu_all_Xraw[np.logical_and(rdu_all_id[:,j]==1., rdu_all_act[:,act]==1.)]
            mat = kwd._dist_matrix(Xhi[:,0,:,0], Xj[:,0,:,0])
            s0 = (mat.sum())/(len(Xhi)*len(Xj))
            
            mat = kwd._dist_matrix(Xhi[:,1,:,0], Xj[:,1,:,0])
            s1 = (mat.sum())/(len(Xhi)*len(Xj))
            dist[i,j] = (s0+s1)/2.
            
            sys.stdout.write("\rj: "+str(j))
            sys.stdout.flush()

        print("")    
        tmp = np.argsort(dist[i])
        tmp = np.where(tmp == test_ids[i])
        ranks[i] = tmp[0][0]
        print("Rank: "+str(ranks[i]))
        if np.argsort(dist[i])[0] in nn:
            nn[np.argsort(dist[i])[0]] += 1
        else:
            nn[np.argsort(dist[i])[0]] = 1 
    print(nn)
    print(ranks)  
    mean += ranks.mean()
    std +=ranks.std()
    print("Mean: "+str(mean/(act+1))+" -- Std = "+str(std/(act+1)))
mean = mean/4.
std = std/4.
print("Final Mean: "+str(mean)+" -- Final Std = "+str(std))

(242, 2, 128, 1)
Test User: 4
j: 23
Rank: 2.0
Test User: 9
j: 23
Rank: 8.0
Test User: 11
j: 23
Rank: 16.0
Test User: 21
j: 23
Rank: 12.0
{2: 2, 19: 2}
[ 2.  8. 16. 12.]
Mean: 9.5 -- Std = 5.172040216394301
Test User: 4
j: 23
Rank: 4.0
Test User: 9
j: 23
Rank: 15.0
Test User: 11
j: 23
Rank: 6.0
Test User: 21
j: 23
Rank: 18.0
{2: 2, 19: 2, 10: 2, 6: 2}
[ 4. 15.  6. 18.]
Mean: 10.125 -- Std = 5.530824855544587
Test User: 4
j: 23
Rank: 8.0
Test User: 9
j: 23
Rank: 14.0
Test User: 11
j: 23
Rank: 17.0
Test User: 21
j: 23
Rank: 6.0
{22: 3, 2: 2, 19: 2, 10: 2, 6: 3}
[ 8. 14. 17.  6.]
Mean: 10.5 -- Std = 5.166236516137962
Test User: 4
j: 23
Rank: 4.0
Test User: 9
j: 23
Rank: 17.0
Test User: 11
j: 23
Rank: 6.0
Test User: 21
j: 23
Rank: 0.0
{2: 2, 19: 2, 6: 3, 22: 3, 7: 1, 10: 2, 21: 3}
[ 4. 17.  6.  0.]
Mean: 9.5625 -- Std = 5.449627783147741
Final Mean: 9.5625 -- Final Std = 5.449627783147741


In [14]:
tX = test_data.copy()

num_comps = 2
print("With "+str(num_comps)+" components:")
 
for i in range(len(tX)):
    tX[i,0,:,0] = ssa_test_0[i].reconstruct(list(range(0,num_comps)))
    tX[i,1,:,0] = ssa_test_1[i].reconstruct(list(range(0,num_comps)))


With 2 components:


In [15]:
dist = np.zeros((4,24))
ranks = np.zeros(4)
kwd = KnnDtw(n_neighbors=10, subsample_step=2)
nn = {} 
mean = 0
std = 0
dd = (w//s)*4

## Labels
rdu_act_X = act_train_labels[::dd]
rdu_act_tX = act_test_labels[::dd]
rdu_id_X = id_train_labels[::dd]
rdu_id_tX = id_test_labels[::dd]

# Test Data: after Transformation
rdu_tX10 = tX[::dd]
print(rdu_tX10.shape)


# Training and Test Data: Raw
rdu_Xraw = Xraw[::dd]
rdu_tXraw = tXraw[::dd]
rdu_all_Xraw = np.append(rdu_Xraw, rdu_tXraw, axis=0)
rdu_all_act = np.append(rdu_act_X,rdu_act_tX,axis=0)
rdu_all_id = np.append(rdu_id_X,rdu_id_tX,axis=0)


import sys
# just considering one activity, like walking. We want to compare data acrros users not activites. 
for act in range(4):
    dic = {}
    for i in range(len(test_ids)):
        print("Test User: "+str(test_ids[i]))
        # Transformed Data
        Xhi = rdu_tX10[np.logical_and(rdu_id_tX[:,test_ids[i]]==1., rdu_act_tX[:,act]==1.)]
        for j in range(24):
            # Raw Data
            Xj = rdu_all_Xraw[np.logical_and(rdu_all_id[:,j]==1., rdu_all_act[:,act]==1.)]
            mat = kwd._dist_matrix(Xhi[:,0,:,0], Xj[:,0,:,0])
            s0 = (mat.sum())/(len(Xhi)*len(Xj))
            
            mat = kwd._dist_matrix(Xhi[:,1,:,0], Xj[:,1,:,0])
            s1 = (mat.sum())/(len(Xhi)*len(Xj))
            dist[i,j] = (s0+s1)/2.
            
            sys.stdout.write("\rj: "+str(j))
            sys.stdout.flush()

        print("")    
        tmp = np.argsort(dist[i])
        tmp = np.where(tmp == test_ids[i])
        ranks[i] = tmp[0][0]
        print("Rank: "+str(ranks[i]))
        if np.argsort(dist[i])[0] in nn:
            nn[np.argsort(dist[i])[0]] += 1
        else:
            nn[np.argsort(dist[i])[0]] = 1 
    print(nn)
    print(ranks)  
    mean += ranks.mean()
    std +=ranks.std()
    print("Mean: "+str(mean/(act+1))+" -- Std = "+str(std/(act+1)))
mean = mean/4.
std = std/4.
print("Final Mean: "+str(mean)+" -- Final Std = "+str(std))

(242, 2, 128, 1)
Test User: 4
j: 23
Rank: 2.0
Test User: 9
j: 23
Rank: 5.0
Test User: 11
j: 23
Rank: 15.0
Test User: 21
j: 23
Rank: 10.0
{2: 1, 19: 3}
[ 2.  5. 15. 10.]
Mean: 8.0 -- Std = 4.949747468305833
Test User: 4
j: 23
Rank: 0.0
Test User: 9
j: 23
Rank: 13.0
Test User: 11
j: 23
Rank: 1.0
Test User: 21
j: 23
Rank: 13.0
{2: 1, 19: 3, 4: 1, 10: 2, 22: 1}
[ 0. 13.  1. 13.]
Mean: 7.375 -- Std = 5.6048697405401455
Test User: 4
j: 23
Rank: 1.0
Test User: 9
j: 23
Rank: 13.0
Test User: 11
j: 23
Rank: 13.0
Test User: 21
j: 23
Rank: 3.0
{2: 1, 19: 3, 4: 1, 22: 4, 20: 1, 10: 2}
[ 1. 13. 13.  3.]
Mean: 7.416666666666667 -- Std = 5.585002578095001
Test User: 4
j: 23
Rank: 0.0
Test User: 9
j: 23
Rank: 14.0
Test User: 11
j: 23
Rank: 7.0
Test User: 21
j: 23
Rank: 0.0
{2: 1, 19: 3, 4: 3, 21: 2, 22: 4, 20: 1, 10: 2}
[ 0. 14.  7.  0.]
Mean: 6.875 -- Std = 5.639775279351738
Final Mean: 6.875 -- Final Std = 5.639775279351738
